<a href="https://colab.research.google.com/github/Sylver640/ADA-Informes/blob/main/Informe_CutRod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Descripción del problema (corte de varillas)
**Entrada:** Largo de varilla $n$, vector de precios por cada medida $i = 1,...,n$.

**Salida:** Retorno máximo $r_n$ que se puede obtener cortando la varilla y vendiendo las partes.

Porque también los algoritmos pueden solucionar situaciones de la cotidianidad, en esta ocasión supongamos que tenemos una compañía que vende **varillas de metal**. Para ello compramos varillas largas y cortas, que luego se cortan en varillas aún más pequeñas, cada medida teniendo su propio valor, tal como se ve en la siguiente tabla de ejemplo:

![image](https://chartreuse-goal-d5c.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F19c92da5-a1d0-4a76-b35c-6ba829aab554%2FUntitled.png?table=block&id=f1440f99-65ce-4bf7-9906-f6980a4b4ae5&spaceId=4f8bebe4-a843-44d2-b6ee-51e2006a90d1&width=1010&userId=&cache=v2)

En ella observamos como va variando el valor de cada corte en un rango $i = [1,10]$, naturalmente incrementando mientras mayor sea el tamaño del corte. Con esta información, ahora debemos encontrar cuál es el retorno máximo que podemos obtener cortando dicha varilla.

![image](https://i.imgur.com/eTMXAYG.gif)

Veremos cómo esto es posible con una forma de programar un tanto interesante: **la programación dinámica**. Este paradigma, al igual que "divide y vencerás", se encargan de resolver el problema combinando las soluciones de cada sub-problema. Sin embargo, algo que diferencia a la primera de esta última es que solo se aplica cuando se **comparten los subsubproblemas**, calculando cada uno tan solo una vez y guardar su solución en una "tabla", lo cual nos evitaría hacer trabajo redundante. En el caso de nuestro problema, observamos que esto sí sucede, puesto que necesitamos el valor previo a la longitud que estamos analizando en dicho momento.

#2. Problema del corte de varilla

##2.1 Código
A continuación, se muestra la implementación del algoritmo que soluciona el problema indicado de dos maneras: recursiva (**Naive Implementation**) y por medio de programación dinámica (**acercamiento bottom-up**).

In [229]:
import random
from termcolor import cprint

comparisons = 0
necessaryCuts = []
subproblems = 0

#Función que retorna el máximo de dos números
def maxSum(a,b):
  if a > b:
    return a
  else:
    return b

#Implementación recursiva del algoritmo
def naiveImplementation(prices, n, verbose):
  #Se llaman a las variables globales
  global comparisons
  global necessaryCuts
  #Se retorna el caso base, es decir, si la longitud del arreglo recibido es menor o igual a 0
  if (n <= 0):
    return 0
  
  maxReturn = -9999999 #Se define a maxReturn como un número muy pequeño para encontrar el máximo con más "facilidad"

  #Se entra en el ciclo para recorrer todo el arreglo
  for i in range(0, n):
    #Se busca el valor máximo entre maxReturn y una llamada recursiva a la función
    maxReturn = maxSum(maxReturn, prices[i] + naiveImplementation(prices, n - i - 1, verbose))
    comparisons+=1

  return maxReturn #Se retorna el retorno máximo encontrado

#Implementación por medio de programación dinámica
def bottomUpImplementation(prices, n, verbose):
  #Llamada a variable global
  global subproblems
  #Arreglo que guardará los retornos máximos anteriores al momento de cada iteración
  maxReturns = [-1]*(n+1)
  maxReturns[0] = 0 #El precio de la longitud 0 se define como $0
  sizes = [-1] * (n+1) #Arreglo que almacenará los cortes necesarios

  #Se va recorriendo 
  for i in range(1, n+1):
    maxReturnValue = -9999999
    for j in range(1, i+1):
      maxReturnValue = maxSum(maxReturnValue, prices[j] + maxReturns[i-j-1])
      sizes[i] = j
    if (maxReturns[i] == -1):
        subproblems+=1
    maxReturns[i] = maxReturnValue
  
  return maxReturns[n], sizes

#Función que se encarga de generar instancias de precios para ser utilizados en cada implementación
def cutrod_instance_generator(N):
  A = []
  prev = 0
  for i in range(N):
    r=random.randint(0,10)
    A.append(prev+r)
    prev+=r
  return A

#Ejemplo
opt = random.randint(1,2)
len = random.randint(6,10)
prices = cutrod_instance_generator(len)
n = random.randint(1,len)
cprint(f"Arreglo de precios: {prices}", 'yellow', attrs=["bold"])
cprint(f"Largo de varilla: {n}")
if opt == 1:
  print("Implementación utilizada: Naive Implementation (recursiva)")
  max = naiveImplementation(prices, n, verbose = False)
  print(f"# Comparaciones = {comparisons}")
if opt == 2:
  print("Implementación utilizada: Bottom-Up (programación dinámica)")
  max, cortes = bottomUpImplementation(prices, n, verbose = False)
  print(f"# Subproblemas solucionados = {subproblems}")

cprint(f"\nRetorno máximo: {max}", 'yellow', attrs=["bold"])
if opt == 2:
  cprint(f"Cantidad de cortes necesarios: {cortes[n]}")

Arreglo de precios: [1, 10, 18, 25, 33, 40, 46, 47, 57, 60]
Largo de varilla: 5
Implementación utilizada: Naive Implementation (recursiva)
# Comparaciones = 31

Retorno máximo: 33


##2.2 Descripción del algoritmo
Tal como se fue mencionado anteriormente, existe más de una forma para solucionar este problema. En nuestro caso, se implementó una solución de dos formas distintas, que se explicarán a continuacion.

###Implementación ingenua
Este método se llama recursivamente solo una vez. Recibe un arreglo o lista de $i$ elementos, o en este caso, precios, y la longitud $n$ de la varilla a cortar. En general, los pasos que sigue esta solución es la siguiente:
1. Recibe los datos requeridos. Si $n$ es menor o igual a $0$, retornamos de inmediato este dato. Éste será nuestro caso base.
2. Se crea una variable $maxReturn$ que irá almacenando los máximos retornos que se vayan encontrando en cada llamada recursiva.
3. Luego, recorremos todo el arreglo de precios y en cada posición buscamos cuál es el mayor retorno en dicha iteración. Sin embargo, en vez de buscar en algún dato anterior o similar, comparamos $maxReturn$ con el precio del corte en esa posición más una llamada recursiva a la función, la cual calculará cada máximo retorno anterior de cada posición.
4. Finalmente, tan solo retornamos el máximo retorno encontrado.

###Acercamiento bottom-up
Al igual que lo descrito anteriormente, se recibe un arreglo o lista que contiene $i$ precios para cada corte, y una longitud $n$ correspondiente a la varilla que deseamos cortar. Puesto que aquí trabajamos bajo el paradigma de la **programación dinámica**, antes de explicar cómo funciona, podemos definir la subestructura óptima del problema. 

Primero, podemos descubrir como nuestra solución óptima, definida por un retorno máximo $r_n$, puede estar dada ya sea por el precio de la varilla completa, o de una subvarilla de precio $p_i$ más el retorno máximo de toda la varilla restante

![image](https://docs.google.com/drawings/d/e/2PACX-1vS1PepvvczFdDNgTY9wP-LyEi5-n8mfg1q1xHeb6ycteXqI0N9vmGjkGG3PI3595JDBChGJeYrVGYP7/pub?w=785&h=407)

Como vemos que este corte es óptimo, en definitiva cada subproblema siguiente, cada uno muy similar al anterior, se verá definida por este valor que nos devolverá la siguiente **función recursiva**:
$r_n=\max\limits_{i=1..n}(p_i+r_{n-i})$, donde $n$ es la longitud máxima de la varilla, $p_i$ el valor del corte encontrado, y $r_{n-i}$ el retorno máximo del valor anterior. Y por último, y que es lo que diferencia a la variable recursiva, es que el **acercamiento bottom-up** guarda los subproblemas anteriores para luego utilizarlos con más facilidad en los casos posteriores. No así la implementación anterior, donde un subproblema se resolvía muchas veces, lo que ya nos entrega una pista de su posible ineficiencia.

Ahora bien, conociendo esta información, podemos describir el funcionamiento de esta implementación de la siguiente forma:
1. Se crea un nuevo arreglo $maxReturns$ que guardará cada nuevo retorno máximo encontrado en cada iteración. Puesto que un corte inexistente tiene un retorno máximo de 0, se define $maxReturns[0] = 0$.
2. Luego, se recorre el arreglo de precios, en donde además se recorre un subarreglo $[1,...,i]$, en donde en cada iteración se irá comparando el valor **maxReturnValue** con $prices[j]$ más un valor anterior del arreglo que almacena los retornos, en donde el mayor se guardará en la posición $maxReturns[i]$.
3. Finalmente, se retorna la $n$-ésima posición del arreglo $maxReturns$, en donde se encontrará el máximo retorno encontrado para dicha longitud.

Para ver paso a paso lo que sucede en cada implementación del algoritmo, `verbose` debe ser igual a `True`.

##2.3 Ejemplo

##2.4 Ejecución del algoritmo paso a paso (`verbose = True`)

#3. Correctitud

#4. Tiempo de ejecución

#5. Experimentos